<a href="https://colab.research.google.com/github/CUNY-SPS-Data-Science-Program/your-bio-AldataSci/blob/main/Story2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Story 2

The following stories contains information about the Consumer Price Index (CPI), the Unemployment Rate and the federal fund Rate, this dataset were downloaded from the FRED economic data and the Bureaur of Labor Statistics (BLS). Here is my attempt at answering the question Has the FED been able to fulfill the mandate given to it by Congress?, visualizations were created using the plotly library in Python.

# Importing The Data
I will first import all three datasets I've found online and import it into my colab notebook.

In [ ]:
import pandas as pd
import plotly.express as px
## This df contains the cpi-u data from the last 25 years..
df = pd.read_csv('https://raw.githubusercontent.com/CUNY-SPS-Data-Science-Program/your-bio-AldataSci/main/Story2/file.csv')

print(df)

       Series ID  Year Period     Label    Value
0    CUSR0000SA0  1998    M01  1998 Jan  162.000
1    CUSR0000SA0  1998    M02  1998 Feb  162.000
2    CUSR0000SA0  1998    M03  1998 Mar  162.000
3    CUSR0000SA0  1998    M04  1998 Apr  162.200
4    CUSR0000SA0  1998    M05  1998 May  162.600
..           ...   ...    ...       ...      ...
303  CUSR0000SA0  2023    M04  2023 Apr  302.918
304  CUSR0000SA0  2023    M05  2023 May  303.294
305  CUSR0000SA0  2023    M06  2023 Jun  303.841
306  CUSR0000SA0  2023    M07  2023 Jul  304.348
307  CUSR0000SA0  2023    M08  2023 Aug  306.269

[308 rows x 5 columns]


The CPI-U data shows the U.S city average CPI for all urban consumers for all items which was seasonally adjusted. The CPI is a measurement of the average change over time in the price paid for the consumer good and services for the last 25 years.

In [ ]:
fedfund = pd.read_csv('https://raw.githubusercontent.com/CUNY-SPS-Data-Science-Program/your-bio-AldataSci/main/Story2/FEDFUNDS.csv')

fedfund

,DATE,FEDFUNDS
0,1997-01-01,5.25
1,1997-02-01,5.19
2,1997-03-01,5.39
3,1997-04-01,5.51
4,1997-05-01,5.50
...,...,...
308,2022-09-01,2.56
309,2022-10-01,3.08
310,2022-11-01,3.78
311,2022-12-01,4.10


 The federal fund rates describes the interest rate that banks charge other banks for lending them caash on an overnight basis. Raising the federal fund rates makes it more expensive for individuals or organizations to borrow funds. Lowering the rates makes it easier for them to engage in borrowing.

In [ ]:
fedfund['DATE']

0      1997-01-01
1      1997-02-01
2      1997-03-01
3      1997-04-01
4      1997-05-01
          ...    
308    2022-09-01
309    2022-10-01
310    2022-11-01
311    2022-12-01
312    2023-01-01
Name: DATE, Length: 313, dtype: object

In [ ]:
unemploy = pd.read_csv('https://raw.githubusercontent.com/CUNY-SPS-Data-Science-Program/your-bio-AldataSci/main/Story2/UNRATE%20(Monthlyy).csv')

unemploy

,DATE,UNRATE
0,1998-01-01,4.6
1,1998-02-01,4.6
2,1998-03-01,4.7
3,1998-04-01,4.3
4,1998-05-01,4.4
...,...,...
303,2023-04-01,3.4
304,2023-05-01,3.7
305,2023-06-01,3.6
306,2023-07-01,3.5


The unemployment rate is the percentage of the labor force without a job, when the economy is in poor shape and jobs are scarce, the unemployment is expected to rise and vice-versa. This includes all industries, occupations,sexes all out of the workforce currently.

## Data Cleaning
I will focus here on polishing the dataset here, to improved readablity and to join tables if I need to. I have removed irrelevant columns within the Cpi data and convert the column into date-time format so that it matches the other columns. For the federal fund table I removed entries from 1997 to better align with the other tables. I've also convert the unemploy date into its proper datetime format and I've inner joined all three tables into one singular tables called result2.

In [ ]:
## I will first clean the df of irrelevant columns.. so I will only have the label column and the value column..
df.rename(columns = {"Label":'DATE',"Value":"CPI-U"},inplace = True)

## Now I will select the relevant columns

Cpi = df[["DATE","CPI-U"]]

Cpi

,DATE,CPI-U
0,1998 Jan,162.000
1,1998 Feb,162.000
2,1998 Mar,162.000
3,1998 Apr,162.200
4,1998 May,162.600
...,...,...
303,2023 Apr,302.918
304,2023 May,303.294
305,2023 Jun,303.841
306,2023 Jul,304.348


In [ ]:
## Now I will convert the DATE columns into DATETIME format..
Cpi["DATE"] = pd.to_datetime(Cpi["DATE"])

Cpi

<ipython-input-6-ee9bcce88250>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cpi["DATE"] = pd.to_datetime(Cpi["DATE"])


,DATE,CPI-U
0,1998-01-01,162.000
1,1998-02-01,162.000
2,1998-03-01,162.000
3,1998-04-01,162.200
4,1998-05-01,162.600
...,...,...
303,2023-04-01,302.918
304,2023-05-01,303.294
305,2023-06-01,303.841
306,2023-07-01,304.348


In [ ]:
## The federal fund rate has data from 1997 so I will remove all entries from 1997

## Create a new entry that contains the year
fedfund["Year"] = fedfund["DATE"].apply(lambda x: x[0:4])

## remove all entries that contain 1997

fedfund['DATE'] = pd.to_datetime(fedfund['DATE'])

fedfund = fedfund[fedfund["Year"] != "1997"]

Fedfund = fedfund[["DATE","FEDFUNDS"]]

Fedfund


,DATE,FEDFUNDS
12,1998-01-01,5.56
13,1998-02-01,5.51
14,1998-03-01,5.49
15,1998-04-01,5.45
16,1998-05-01,5.49
...,...,...
308,2022-09-01,2.56
309,2022-10-01,3.08
310,2022-11-01,3.78
311,2022-12-01,4.10


In [ ]:
## Let me convert the unemployment rate table into proper datetime format before merging it to the final table

Unemploy = unemploy

Unemploy['DATE'] = pd.to_datetime(Unemploy['DATE'])

Unemploy

,DATE,UNRATE
0,1998-01-01,4.6
1,1998-02-01,4.6
2,1998-03-01,4.7
3,1998-04-01,4.3
4,1998-05-01,4.4
...,...,...
303,2023-04-01,3.4
304,2023-05-01,3.7
305,2023-06-01,3.6
306,2023-07-01,3.5


In [ ]:
result = pd.merge(Fedfund,Cpi,on = 'DATE',how = 'inner')

## Create another join with the unemploy data..

result2 = pd.merge(result,Unemploy,on = 'DATE',how = 'inner')

## Now we have our final table..
result2

,DATE,FEDFUNDS,CPI-U,UNRATE
0,1998-01-01,5.56,162.000,4.6
1,1998-02-01,5.51,162.000,4.6
2,1998-03-01,5.49,162.000,4.7
3,1998-04-01,5.45,162.200,4.3
4,1998-05-01,5.49,162.600,4.4
...,...,...,...,...
296,2022-09-01,2.56,296.539,3.5
297,2022-10-01,3.08,297.987,3.7
298,2022-11-01,3.78,298.598,3.6
299,2022-12-01,4.10,298.990,3.5


# Data Visualizations
Now I will create some data visualizations from the joined data.

In [ ]:
fig = px.line(result2,x = "DATE", y = "FEDFUNDS")

fig.update_layout(title = "Federal Fund Rate For the Last 25 Years",xaxis_title = 'Year',yaxis_title = 'Federal Fund Percent Rate')

In [ ]:
fig2 = px.line(result2,x = "DATE",y = "CPI-U")

fig2.update_layout(title = "CPI-U Rate",xaxis_title = 'Date',yaxis_title = 'CPI-U Rate')

In [ ]:
fig3 = px.line(result2, x= "DATE", y = "UNRATE")

fig3.update_layout(title = "Unemployment Rate For the last 25 years",xaxis_title = 'Date',yaxis_title = 'Unemployment Rate')

# Overview:

Each plot seems to reflect a different story,however we can see various fluctuations in the plots. The CPI rate has been steadily increasing over the years with minor hiccups which tells us that inflation has been increasing thorughout the years. The unemployment rate seems to decrease during the first half of the decade but rose suddenly around 8-10% during 2008 which was an effect of the 2008 housing bubble crash, and a sharp increase from 3% to 14% in 2020 which was the start of the pandemic. These events seem to have influenced the fed fund rate as we see sharp decreases in fed rates during 2008 and the 2020s which was a response in order to stimulate the economy.

In [ ]:
fig4 = px.line(result2, x= "DATE", y = ["UNRATE","FEDFUNDS"])

fig4.update_layout(title = "Rate Trends Over Time",xaxis_title = 'Years',yaxis_title = 'Percentage (%)',)

fig4.show()

I've plotted the CPI Rate and The unemployment Rate on the same plot, it appears that the relationship between the two are negatively related. It seems that when the unemployment rate is decreasing the federal reserve decides to increase the fund rate to help reduce rising inflation and curb consumer spending to slow down the economy.On the other hand, when unemployment is increasing the federal fund rate is decreasing as shown during the 2008 housing crash and the pandemic in 2020 where unemployment levels were high and as a result, the federal reserve decided to signficantly reduce interest rates in an attempt to stimulate the economy. In simpler terms, if unemployment rates are low increase the federal fund rate and when unemployment rates are high, decrease the federal fund rate.

#Visualizations Two
I decided to create a scatterplot to determine if there are any relationships between the variables.

In [ ]:
## Create a scatterplot potentially..?
px.scatter(result2,x = 'CPI-U',y = "UNRATE", title = "Relationship between Unemployment Rate and Cpi Rate is unclear<br><sup>PCI Rate and Unemployment Rate appear to be non-linear</sup>",opacity = 0.4)

In [ ]:
px.scatter(result2,x = 'CPI-U',y = "FEDFUNDS", title = "Relationship between CPI and Federal Fund Rate <br><sup>Inflation is rising no matter how much the FFR is being adjusted </sup>",opacity = 0.4)

In [ ]:
px.scatter(result2,x = "UNRATE",y = "FEDFUNDS", title = "Higher unemployment results in lower Rates <br><sup>High unemployment leads to reduced FF Rates</sup>",opacity = 0.4)

The scatterplots all seems to depict different narratives regarding the state of the economy for the past 25 years. But one thing is clear, regardless what the Federal Reserve does to adjust the Federal Fund Rate inflation is rising regardless. However, the unemployment rate seems to decrease when the federal fund rate is sharpily reduced. Interestingly, inflation goes up regardless if the unemployment rate is increasing or decreasing.

# Conclusion:
Has The Fed Been able to fufill the mandate from Congress?

My answer **is it depends**.. from the overlayed line graphs and the scatterplots we can see that adjusting the federal fund rate has been able to influence the unemployment rate. We see that low unemployment rates lead to higher rates while high unemployment levels lead to lower rates. So the fed has been able to maintain low unemployment. However, we can see from the Cpi metric that for the last 25 years inflation has been steadily rising despite some few hiccups and no matter how much the federal fund rate is adjusted inflation keeps rising, so on that half the fed has failed. **The Fed has fufilled half of the mandate they are able to maintain low unemployment but can not combat high inflation.**

# Work-Cited

Here are the links from the data I have sourced..

This is the Cpi data I've found from Bureau of Labor Statistics.

“BLS Data Viewer.” U.S. Bureau of Labor Statistics, U.S. Bureau of Labor Statistics, [beta.bls.gov/dataViewer/view/timeseries/CUSR0000SA0](https://). Accessed 21 Sept. 2023.

This is the Unemployment Data I've found from the FRED website

“Unemployment Rate.” FRED, 1 Sept. 2023, [fred.stlouisfed.org/series/UNRATE#0](https://).

The Federal Fund Rate also from the Fred website

“Federal Funds Effective Rate.” FRED, 1 Sept. 2023, [fred.stlouisfed.org/series/FEDFUNDS](https://).

